In [ ]:
import re
import json
import sqlite3
import pandas as pd
import openpyxl
import requests

def scrape_election_data_from_view_source(state_name):
    # Construct the URL
    url = f"https://www.nbcnews.com/politics/2024-elections/{state_name}-president-results"

    # Fetch the page source
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data for {state_name}: {response.status_code}")
        return []

    # Look for JSON-like structure in the page source
    try:
        # Regex pattern to extract the JSON array with county-level data
        json_pattern = re.compile(r'\[\{"name":"[^"]+","percentIn":\d+\.?\d*,"votes":\d+,"candidates":\[.*?\]\}\]')
        match = json_pattern.search(response.text)
        if not match:
            print(f"No JSON data found for {state_name}")
            return []

        # Parse the JSON
        data = json.loads(match.group(0))
        print(f"Successfully fetched data for {state_name}")
        return data
    except Exception as e:
        print(f"Error parsing data for {state_name}: {e}")
        return []

def save_to_sqlite(db_name, state_name, data):
    # Connect to SQLite database (or create it)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create tables if they don't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS counties (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            state TEXT,
            county_name TEXT,
            percent_in REAL,
            total_votes INTEGER
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS candidates (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            county_id INTEGER,
            name TEXT,
            party TEXT,
            votes INTEGER,
            percent_vote REAL,
            FOREIGN KEY (county_id) REFERENCES counties (id)
        )
    ''')

    # Insert data into tables
    for county in data:
        cursor.execute('''
            INSERT INTO counties (state, county_name, percent_in, total_votes)
            VALUES (?, ?, ?, ?)
        ''', (state_name, county['name'], county['percentIn'], county['votes']))
        
        county_id = cursor.lastrowid  # Get the last inserted county ID

        for candidate in county['candidates']:
            # Normalize party name if needed (e.g., "gop" to "Republican")
            party = candidate['party']
            if party.lower() == 'gop':
                party = 'Republican'
            cursor.execute('''
                INSERT INTO candidates (county_id, name, party, votes, percent_vote)
                VALUES (?, ?, ?, ?, ?)
            ''', (county_id, candidate['name'], party, candidate['votes'], candidate['percentVote']))

    # Commit and close connection
    conn.commit()
    conn.close()
    print(f"Data for {state_name} saved to {db_name}.")

def scrape_and_save_all_states():
    # List of all states and D.C., formatted for URLs
    states = [
        "alabama", "alaska", "arizona", "arkansas", "california", "colorado",
        "connecticut", "delaware", "district-of-columbia", "florida", "georgia",
        "hawaii", "idaho", "illinois", "indiana", "iowa", "kansas", "kentucky",
        "louisiana", "maine", "maryland", "massachusetts", "michigan", "minnesota",
        "mississippi", "missouri", "montana", "nebraska", "nevada", "new-hampshire",
        "new-jersey", "new-mexico", "new-york", "north-carolina", "north-dakota",
        "ohio", "oklahoma", "oregon", "pennsylvania", "rhode-island",
        "south-carolina", "south-dakota", "tennessee", "texas", "utah", "vermont",
        "virginia", "washington", "west-virginia", "wisconsin", "wyoming"
    ]

    # Database name
    db_name = "election_results.db"

    for state in states:
        print(f"Scraping data for {state}...")
        data = scrape_election_data_from_view_source(state)
        if data:
            save_to_sqlite(db_name, state, data)

def query_election_data(db_name, limit=100):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)

    # Define the SQL query with a limit
    query = f"""
        SELECT c.state, c.county_name, ca.name AS candidate_name, ca.party, ca.votes, ca.percent_vote
        FROM counties c
        JOIN candidates ca ON c.id = ca.county_id
        LIMIT {limit};
    """

    # Execute the query and load into a DataFrame
    df = pd.read_sql_query(query, conn)

    # Close the database connection
    conn.close()

    # Display the DataFrame
    print(df)

def save_csv_to_sqlite(db_name, dataframe, table_name, schema_sql):
    # Connect to SQLite database (or create it)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute(schema_sql)
    conn.commit()

    # Insert cleaned data into the table
    dataframe.to_sql(table_name, conn, if_exists='append', index=False)

    # Commit and close the connection
    conn.commit()
    conn.close()
    print(f"{table_name} data successfully added to the database.")

def query_historical_data(db_name, limit=10):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)

    # Define the SQL query with a limit
    query = f"""
        SELECT * 
        FROM historical_election_results
        LIMIT {limit};
    """

    # Execute the query and load into a DataFrame
    df = pd.read_sql_query(query, conn)

    # Close the database connection
    conn.close()

    # Display the DataFrame
    print(df)

# ----------------------------------------
# Main script execution starts here
# ----------------------------------------

db_name = "election_results.db"

# Scrape 2024 data and save to DB
scrape_and_save_all_states()

# Query some election data (example)
query_election_data(db_name, limit=100)

# Load and clean the historical dataset
election_data = pd.read_csv("countypres_2000-2020.csv")
election_data_cleaned = election_data[
    ['year', 'state', 'state_po', 'county_name', 'county_fips', 'candidate', 
     'party', 'candidatevotes', 'totalvotes']
].dropna()

# Rename columns for clarity
election_data_cleaned = election_data_cleaned.rename(columns={
    'state_po': 'state_abbreviation',
    'county_fips': 'fips_code',
    'candidatevotes': 'votes_candidate',
    'totalvotes': 'votes_total'
})

# Save historical election results to DB
schema_historical = '''
    CREATE TABLE IF NOT EXISTS historical_election_results (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        year INTEGER,
        state TEXT,
        state_abbreviation TEXT,
        county_name TEXT,
        fips_code REAL,
        candidate TEXT,
        party TEXT,
        votes_candidate INTEGER,
        votes_total INTEGER
    )
'''
save_csv_to_sqlite(db_name, election_data_cleaned, 'historical_election_results', schema_historical)

# Query some historical data (example)
query_historical_data(db_name, limit=100)

# Add unemployment data
unemployment_data = pd.read_csv("Unemployment(UnemploymentMedianIncome).csv")
unemployment_cols = [col for col in unemployment_data.columns if 'Unemployment_rate_' in col]

unemp_long = unemployment_data.melt(
    id_vars=['FIPS_Code','State','Area_Name'],
    value_vars=unemployment_cols,
    var_name='year_var',
    value_name='unemp_rate'
)

unemp_long['year'] = unemp_long['year_var'].str.extract(r'(\d{4})').astype(int)
unemp_long = unemp_long.dropna(subset=['unemp_rate'])
unemp_long = unemp_long.rename(columns={'FIPS_Code': 'fips_code'})

schema_unemployment = '''
    CREATE TABLE IF NOT EXISTS unemployment_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fips_code INTEGER,
        State TEXT,
        Area_Name TEXT,
        year_var TEXT,
        unemp_rate REAL,
        year INTEGER
    )
'''

save_csv_to_sqlite(db_name, unemp_long, 'unemployment_data', schema_unemployment)

# Add population data
population_data = pd.read_csv('combined_population_with_fips.csv')
pop_cols = [col for col in population_data.columns if 'POPESTIMATE' in col]

pop_long = population_data.melt(
    id_vars=['STATE','COUNTY','STNAME','FIPS'],
    value_vars=pop_cols,
    var_name='pop_year_var',
    value_name='population_est'
)

pop_long['year'] = pop_long['pop_year_var'].str.extract(r'(\d{4})').astype(int)
pop_long = pop_long.dropna(subset=['population_est'])
pop_long['fips_code'] = pd.to_numeric(pop_long['FIPS'], errors='coerce')
pop_long = pop_long[['fips_code','year','population_est']]

schema_population = '''
    CREATE TABLE IF NOT EXISTS population_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fips_code INTEGER,
        year INTEGER,
        population_est INTEGER
    )
'''

save_csv_to_sqlite(db_name, pop_long, 'population_data', schema_population)

# Add education data
file_path = "Education.xlsx"
education_df = pd.read_excel(file_path, engine='openpyxl', header=3)
education_df = education_df.rename(columns={'FIPS Code': 'fips_code'})

# Columns to select for bachelor's degree or higher
bachelors_cols = [
    "Percent of adults with a bachelor's degree or higher, 1990",
    "Percent of adults with a bachelor's degree or higher, 2000",
    "Percent of adults with a bachelor's degree or higher, 2008-12",
    "Percent of adults with a bachelor's degree or higher, 2018-22"
]

# Columns to select for high school diploma only
hs_cols = [
    "Percent of adults with a high school diploma only, 1990",
    "Percent of adults with a high school diploma only, 2000",
    "Percent of adults with a high school diploma only, 2008-12",
    "Percent of adults with a high school diploma only, 2018-22"
]

edu_cols = ['fips_code'] + bachelors_cols + hs_cols
education_df = education_df[edu_cols]

# Rename columns for clarity
education_df = education_df.rename(columns={
    "Percent of adults with a bachelor's degree or higher, 1990": "bach_1990",
    "Percent of adults with a bachelor's degree or higher, 2000": "bach_2000",
    "Percent of adults with a bachelor's degree or higher, 2008-12": "bach_2008_12",
    "Percent of adults with a bachelor's degree or higher, 2018-22": "bach_2018_22",
    "Percent of adults with a high school diploma only, 1990": "hs_1990",
    "Percent of adults with a high school diploma only, 2000": "hs_2000",
    "Percent of adults with a high school diploma only, 2008-12": "hs_2008_12",
    "Percent of adults with a high school diploma only, 2018-22": "hs_2018_22"
})

schema_education = '''
    CREATE TABLE IF NOT EXISTS education_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fips_code INTEGER,
        bach_1990 REAL,
        bach_2000 REAL,
        bach_2008_12 REAL,
        bach_2018_22 REAL,
        hs_1990 REAL,
        hs_2000 REAL,
        hs_2008_12 REAL,
        hs_2018_22 REAL
    )
'''

save_csv_to_sqlite(db_name, education_df, 'education_data', schema_education)

print("All data has been successfully saved to the database. You can now run queries to explore the data.")


Scraping data for alabama...
Successfully fetched data for alabama
Data for alabama saved to election_results.db.
Scraping data for alaska...
Successfully fetched data for alaska
Data for alaska saved to election_results.db.
Scraping data for arizona...
Successfully fetched data for arizona
Data for arizona saved to election_results.db.
Scraping data for arkansas...
Successfully fetched data for arkansas
Data for arkansas saved to election_results.db.
Scraping data for california...
Successfully fetched data for california
Data for california saved to election_results.db.
Scraping data for colorado...
Successfully fetched data for colorado
Data for colorado saved to election_results.db.
Scraping data for connecticut...
Successfully fetched data for connecticut
Data for connecticut saved to election_results.db.
Scraping data for delaware...
Successfully fetched data for delaware
Data for delaware saved to election_results.db.
Scraping data for district-of-columbia...
Successfully fetche